In [4]:
import os
import pandas as pd
import re
import numpy as np
os.getcwd()
df = pd.read_csv("../data/attacks.csv", encoding="Latin1")

# Analizamos la información

In [5]:
df.shape

(25723, 24)

In [6]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [7]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

# Comprobamos valores nulos

In [8]:
df.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

La mayoría de columnas tiene unos 19.000 valores nulos aproximadamente, por tanto decidimos eliminar aquellas columnas que tengan más de 20.000 valores nulos

In [10]:
morethan20000nulls = df.isnull().sum()[df.isnull().sum() > 20000].index
morethan20000nulls

Index(['Age', 'Time', 'Species ', 'Unnamed: 22', 'Unnamed: 23'], dtype='object')

In [11]:
#Definimos función para eliminar columnas
def dropcolumns (df, list):
    df_clean = df.drop(columns=list)
    return df_clean

In [12]:
df_clean = dropcolumns (df, morethan20000nulls)
df_clean.shape

(25723, 19)

Ahora veremos si hay filas en las que todos sus datos sean nulos y las eliminaremos

In [13]:
df_clean = df_clean.dropna(how="all")
df_clean.shape

(8703, 19)

# Columnas que no vamos a utilizar

A pesar de que hemos reducido bastante la información, seguimos teniendo columnas que no vamos a necesitar, por lo que las eliminaremos también

In [14]:
df_clean.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Injury', 'Fatal (Y/N)',
       'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [15]:
unusefull_columns = ['Case Number', 'Name', 'Sex ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order']
df_clean = dropcolumns (df_clean, unusefull_columns)
df_clean.shape

(8703, 9)

Y volvemos a comprobar si hay filas en las que todos los valores sean nulos y las eliminamos

In [17]:
df_clean = df_clean.dropna(how="all")
df_clean.shape

(6302, 9)

# Analizamos columnas

Ahora que ya hemos elimados los valores nulos, y aquellas columnas que no vamos a necesitar podemos comenzar a ver información más detallada de cada una de las columnas

In [19]:
df_clean.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Injury', 'Fatal (Y/N)'],
      dtype='object')

In [44]:
#Columna "Year"
df_clean["Year"].value_counts().reset_index().sort_values('index', ascending=False)

,index,Year
37,2018,53
1,2017,136
2,2016,130
0,2015,143
5,2014,127
...,...,...
248,1543,1
211,500,1
241,77,1
247,5,1


Podemos ver que en el año 2018 hay muchos menos registros que en años anteriores, es posible que el año no esté completo, lo comprobaremos analizando la columna "Date"

In [22]:
list(df_clean["Date"].value_counts().items())

[('1957', 11),
 ('1942', 9),
 ('1956', 8),
 ('1958', 7),
 ('1950', 7),
 ('1941', 7),
 ('1949', 6),
 ('No date', 6),
 ('1940', 5),
 ('28-Jul-1995', 5),
 ('1959', 5),
 ('05-Oct-2003', 5),
 ('Oct-1960', 5),
 ('1955', 5),
 ('1954', 5),
 ('Aug-1956', 5),
 ('No date, Before 1963', 5),
 ('1970s', 5),
 ('12-Apr-2001', 5),
 ('Reported 10-Oct-1906', 4),
 ('1876', 4),
 ('1904', 4),
 ('20-Sep-2015', 4),
 ('1960s', 4),
 ('09-Jan-2010', 4),
 ('27-Jul-1952', 4),
 ('1898', 4),
 ('1995', 4),
 ('Before 1958', 4),
 ('14-Jun-2012', 4),
 ('1952', 4),
 ('1960', 4),
 ('1961', 4),
 ('1938', 4),
 ('15-Apr-2018', 4),
 ('09-Jul-1994', 4),
 ('27-Dec-2008', 4),
 ('Before 1906', 4),
 ('29-Apr-2017', 4),
 ('1945', 4),
 ('23-Jan-1970', 4),
 ('28-Dec-2014', 4),
 ('08-Aug-1899', 3),
 ('1913', 3),
 ('23-Oct-2017', 3),
 ('31-Oct-2003', 3),
 ('26-Jun-2015', 3),
 ('1971', 3),
 ('17-Aug-2011', 3),
 ('12-Jul-1916', 3),
 ('19-Aug-2007', 3),
 (' 01-Sep-2013', 3),
 ('08-Mar-1992', 3),
 ('11-Jan-1962', 3),
 ('13-Mar-1977', 3),
 

Observamos que en algunas celdas la información es incompleta, ya que muestra solo el año, o que incluye la palabra "Reported", eliminaremos información no necesaria y nos quedaremos solo con las celdas que contienen la fecha completa haciendo uso de Regex

In [23]:
#Usamos Regex en la columna "Date" para quedarnos con las celdas que cumplan el formato dd/mm/yyyy
def datecolumn(columnseries):
    reg = re.findall(r'[0-9]+\-[a-zA-Z]*\-[0-9]{4}', str(columnseries))
    if reg:
        return reg[0]
    else:
        return None
df_clean["Date"].apply(datecolumn)

0       25-Jun-2018
1       18-Jun-2018
2       09-Jun-2018
3       08-Jun-2018
4       04-Jun-2018
           ...     
6297           None
6298           None
6299           None
6300           None
6301           None
Name: Date, Length: 6302, dtype: object

In [24]:
#Creamos un nueva columna llamada "Date_ok"
df_clean["Date_ok"] = df_clean["Date"].apply(datecolumn)

In [25]:
#Columna "Type"
list(df_clean["Type"].value_counts().items())

[('Unprovoked', 4595),
 ('Provoked', 574),
 ('Invalid', 547),
 ('Sea Disaster', 239),
 ('Boating', 203),
 ('Boat', 137),
 ('Questionable', 2),
 ('Boatomg', 1)]

Parece que los valores "Boating", "Boat" y "Boatomg" hacen referencia a lo mismo, por lo que podemos agruparlos con el mismo nombre

In [26]:
#Comprobamos si existen valores nulos
df_clean["Type"].isnull().sum()

4

In [27]:
df_clean[df_clean.Type.isnull()][["Year","Country","Type"]]

,Year,Country,Type
85,2017.0,SAMOA,NaN
382,2015.0,AUSTRALIA,NaN
4867,1936.0,VIETNAM,NaN
5705,1890.0,CEYLON,NaN


In [28]:
#Rellenamos los 4 valores nulos como "Sin registros"
df_clean.loc[df_clean.Type.isnull(), "Type"] = ["Sin registros","Sin registros","Sin registros","Sin registros"]

In [29]:
#Agrupamos "Boating", "Boat" y "Boatomg" bajo "Boating"
df_clean.loc[df_clean["Type"].str.startswith("B"),"Type"] = "Boating"

In [30]:
df_clean["Type"].value_counts()

Unprovoked       4595
Provoked          574
Invalid           547
Boating           341
Sea Disaster      239
Sin registros       4
Questionable        2
Name: Type, dtype: int64

In [31]:
#Columna "Activity"
list(df_clean["Activity"].value_counts().items())

[('Surfing', 971),
 ('Swimming', 869),
 ('Fishing', 431),
 ('Spearfishing', 333),
 ('Bathing', 162),
 ('Wading', 149),
 ('Diving', 127),
 ('Standing', 99),
 ('Snorkeling', 89),
 ('Scuba diving', 76),
 ('Body boarding', 61),
 ('Body surfing', 49),
 ('Swimming ', 47),
 ('Kayaking', 33),
 ('Fell overboard', 32),
 ('Pearl diving', 32),
 ('Treading water', 32),
 ('Boogie boarding', 29),
 ('Free diving', 29),
 ('Windsurfing', 19),
 ('Walking', 17),
 ('Boogie Boarding', 16),
 ('Shark fishing', 15),
 ('Floating', 14),
 ('Canoeing', 13),
 ('Fishing ', 13),
 ('Rowing', 12),
 ('Surf fishing', 12),
 ('Surf skiing', 12),
 ('Surf-skiing', 12),
 ('Fishing for sharks', 11),
 ('Kayak Fishing', 11),
 ('Scuba Diving', 10),
 ('Freediving', 10),
 ('Sponge diving', 10),
 ('Sitting on surfboard', 9),
 ('Diving for trochus', 9),
 ('Sailing', 9),
 ('Fell into the water', 9),
 ('Sea disaster', 8),
 ('Paddle boarding', 8),
 ('Boating', 7),
 ('Playing', 7),
 ('Free diving for abalone', 7),
 ('Spearfishing ', 7),


In [32]:
df_clean["Activity"].isnull().sum()

544

In [33]:
df_clean[df_clean.Activity.isnull()][["Year","Country","Type"]]

,Year,Country,Type
47,2018.0,AUSTRALIA,Unprovoked
108,2017.0,USA,Unprovoked
112,2017.0,USA,Unprovoked
114,2017.0,USA,Unprovoked
154,2017.0,USA,Unprovoked
...,...,...,...
6269,0.0,PACIFIC OCEAN,Sea Disaster
6277,0.0,AUSTRALIA,Unprovoked
6282,0.0,AUSTRALIA,Unprovoked
6292,0.0,USA,Unprovoked


In [34]:
#Definimos los valores nulos como "NoActivity"
df_clean["Activity"] = df_clean.Activity.fillna("NoActivity")

In [35]:
#Comprobamos que ya no hay valores nulos
df_clean["Activity"].isnull().sum()

0

En la columna "Actividades" podemos comprobar como hay varias que hacen referencia a Surfing con distintos nombres, trataremos de agruparlas todas bajo la denominación "Surfing"

In [36]:
#Aquellas que comienzan con la palabra Surf "S" mayúscula
df_clean.loc[df_clean["Activity"].str.startswith("Surf"),"Activity"] = "Surfing"
#quellas que comienzan con la palabra Surf "s" minúscula
df_clean.loc[df_clean["Activity"].str.startswith("surf"),"Activity"] = "Surfing"
#Aquellas que acaban con la palabra Surfing "S" mayúscula
df_clean.loc[df_clean["Activity"].str.endswith("surfing"),"Activity"] = "Surfing"
#Aquellas que acaban con la palabra Surfing "s" minúscula
df_clean.loc[df_clean["Activity"].str.endswith("Surfing"),"Activity"] = "Surfing"

In [37]:
list(df_clean["Activity"].value_counts().items())

[('Surfing', 1166),
 ('Swimming', 869),
 ('NoActivity', 544),
 ('Fishing', 431),
 ('Spearfishing', 333),
 ('Bathing', 162),
 ('Wading', 149),
 ('Diving', 127),
 ('Standing', 99),
 ('Snorkeling', 89),
 ('Scuba diving', 76),
 ('Body boarding', 61),
 ('Swimming ', 47),
 ('Kayaking', 33),
 ('Treading water', 32),
 ('Fell overboard', 32),
 ('Pearl diving', 32),
 ('Free diving', 29),
 ('Boogie boarding', 29),
 ('Walking', 17),
 ('Boogie Boarding', 16),
 ('Shark fishing', 15),
 ('Floating', 14),
 ('Canoeing', 13),
 ('Fishing ', 13),
 ('Rowing', 12),
 ('Kayak Fishing', 11),
 ('Fishing for sharks', 11),
 ('Freediving', 10),
 ('Scuba Diving', 10),
 ('Sponge diving', 10),
 ('Sitting on surfboard', 9),
 ('Fell into the water', 9),
 ('Diving for trochus', 9),
 ('Sailing', 9),
 ('Paddle boarding', 8),
 ('Sea disaster', 8),
 ('Boating', 7),
 ('Playing', 7),
 ('Free diving for abalone', 7),
 ('Floating on his back', 7),
 ('Spearfishing ', 7),
 ('Diving for abalone', 7),
 ('Skindiving', 7),
 ('Stand-Up

In [38]:
df_clean.head()

,Date,Year,Type,Country,Area,Location,Activity,Injury,Fatal (Y/N),Date_ok
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N,25-Jun-2018
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N,18-Jun-2018
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N,09-Jun-2018
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,08-Jun-2018
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,04-Jun-2018


In [39]:
#Reordenamos las columnas, colocando la nueva columna "Date_ok" al principio
colnames = df_clean.columns.tolist()
colnames = colnames[-1:] + colnames [:-1]
df_clean = df_clean[colnames]

In [40]:
df_clean = df_clean.drop(["Date"], axis=1)

In [41]:
df_clean.head()

,Date_ok,Year,Type,Country,Area,Location,Activity,Injury,Fatal (Y/N)
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N


# Guardamos el CSV limpio

In [42]:
df_clean.to_csv("../data/attacks_clean.csv")